# SWaT Anomaly Detection with OmniAnomaly

by dongmin kim (tommy.dm.kim@gmail.com)

In [1]:
import sys
HOME = "../../../"
sys.path.append(HOME) # repo home

In [2]:
from exp_helpers.utils import SEED_everything
SEED_everything(42)

In [3]:
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import easydict

## Configure settings

In [4]:
args = easydict.EasyDict({
    # general
    "dataset": "SWaT",
    "batch_size": 64,
    "lr": 1e-03,
    "window_size": 100,
    "epochs": 30,
    "use_tqdm": True,
    "tqdmopt": "notebook",
    "load_pretrained": False,

    # model-specific (OmniAnomaly)
    "model": "OmniAnomaly",
    "latent_dim": 128,
    "num_layers": 1,
    "z_dim": 3,
    "dense_dim": 10,

    # others
    "device": torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'),
})

In [5]:
config = ""
config += f"_dataset_{args.dataset}" \
          f"_batch_size_{args.batch_size}" \
          f"_lr_{args.lr}" \
          f"_window_size_{args.window_size}" \

config += f"_model_{args.model}" \
          f"_latent_dim_{args.latent_dim}" \
          f"_num_layers_{args.num_layers}" \
          f"_z_dim_{args.z_dim}" \
          f"_dense_dim_{args.dense_dim}"

args.config = config

In [6]:
# PATH to save model
os.makedirs(os.path.join(HOME, "models", "checkpoints", f"{args.config}"), exist_ok=True)
args.model_path = os.path.join(HOME, "models", "checkpoints", f"{args.config}", "model.pt")

# PATH to save result figures
os.makedirs(os.path.join(HOME, "results"), exist_ok=True)
os.makedirs(os.path.join(HOME, "results", args.model), exist_ok=True)
os.makedirs(os.path.join(HOME, "results", args.model, args.config), exist_ok=True)
args.result_path = os.path.join(HOME, "results", args.model, args.config)
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(args.model_path)
print(args.result_path)

../../../models/checkpoints/_dataset_SWaT_batch_size_64_lr_0.001_window_size_100_model_OmniAnomaly_latent_dim_128_num_layers_1_z_dim_3_dense_dim_10/model.pt
../../../results/OmniAnomaly/_dataset_SWaT_batch_size_64_lr_0.001_window_size_100_model_OmniAnomaly_latent_dim_128_num_layers_1_z_dim_3_dense_dim_10


## Load data

In [7]:
from data.load_data import load_data
from data.dataset import get_dataset

train_x, train_y, test_x, test_y = load_data(args.dataset, HOME)

train_dataset = get_dataset(train_x, train_y, window_size = args.window_size, dataset_type=args.dataset)
test_dataset = get_dataset(test_x, test_y, window_size = args.window_size, dataset_type=args.dataset)

train_loader = torch.utils.data.DataLoader(
                 dataset=train_dataset,
                 batch_size=args.batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_dataset,
                batch_size=args.batch_size,
                shuffle=False)

args.input_dim = train_x.shape[1]

Reading SWaT...
current location: /home/nas3_userJ/dmkim/TSAD/notebooks/Modeling/SWaT
home dir: ../../../
Loading complete.


## Model

In [8]:
from models.utils import prepare_model, prepare_loss_fn
print(f"setting model to {args.device}...")
model = prepare_model(args)
model.to(args.device)

setting model to cuda...


OmniAnomaly(
  (qnet): Qnet(
    (gru): GRU(51, 128, batch_first=True)
    (dense): Linear(in_features=131, out_features=10, bias=True)
    (linear_mu): Linear(in_features=10, out_features=3, bias=True)
    (linear_sigma): Linear(in_features=10, out_features=3, bias=True)
    (softplus): Softplus(beta=1, threshold=20)
    (pnf): PlanarNormalizingFlow(
      (linear): Linear(in_features=3, out_features=3, bias=True)
      (tanh): Tanh()
    )
  )
  (pnet): Pnet(
    (gru): GRU(3, 128, batch_first=True)
    (LGSSM): Linear(in_features=3, out_features=3, bias=True)
    (dense): Linear(in_features=128, out_features=10, bias=True)
    (linear_mu): Linear(in_features=10, out_features=51, bias=True)
    (linear_sigma): Linear(in_features=10, out_features=51, bias=True)
    (softplus): Softplus(beta=1, threshold=20)
  )
)

In [ ]:
from exp_helpers.exp_OmniAnomaly import OmniAnomaly_Trainer

if args.load_pretrained is True:
    print(f"loading pretrained model at {args.model_path}...")
    best_model = model
    best_model.load_state_dict(torch.load(args.model_path))
    best_model.to(args.device)
    print("done")
else:
    print("start training...")
    optimizer = torch.optim.Adam(params=model.parameters(), lr = args.lr)
    loss_fn = prepare_loss_fn(args)

    trainer = OmniAnomaly_Trainer(
        args=args,
        model=model,
        train_loader=train_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
    )

    best_model = trainer.train()

start training...
{'dataset': 'SWaT', 'batch_size': 64, 'lr': 0.001, 'window_size': 100, 'epochs': 30, 'use_tqdm': True, 'tqdmopt': 'notebook', 'load_pretrained': False, 'model': 'OmniAnomaly', 'latent_dim': 128, 'num_layers': 1, 'z_dim': 3, 'dense_dim': 10, 'device': device(type='cuda'), 'config': '_dataset_SWaT_batch_size_64_lr_0.001_window_size_100_model_OmniAnomaly_latent_dim_128_num_layers_1_z_dim_3_dense_dim_10', 'model_path': '../../../models/checkpoints/_dataset_SWaT_batch_size_64_lr_0.001_window_size_100_model_OmniAnomaly_latent_dim_128_num_layers_1_z_dim_3_dense_dim_10/model.pt', 'result_path': '../../../results/OmniAnomaly/_dataset_SWaT_batch_size_64_lr_0.001_window_size_100_model_OmniAnomaly_latent_dim_128_num_layers_1_z_dim_3_dense_dim_10', 'input_dim': 51}


  0%|          | 0/30 [00:00<?, ?it/s]

training:   0%|          | 0/7761 [00:00<?, ?it/s]

training:   0%|          | 0/7761 [00:00<?, ?it/s]

training:   0%|          | 0/7761 [00:00<?, ?it/s]

In [ ]:
from exp_helpers.exp_OmniAnomaly import OmniAnomaly_Tester

tester = OmniAnomaly_Tester(
    args = args,
    model = best_model,
    train_loader = train_loader,
    test_loader = test_loader,
)

In [ ]:
from data.load_data import load_anomaly_intervals

In [ ]:
anomaly_scores = tester.get_anomaly_score()

In [ ]:
intervals = load_anomaly_intervals(anomaly_labels = test_y, window_size = args.window_size)

In [ ]:
best_threshold = tester.regular_thresholding(test_y, anomaly_scores, intervals, num_candidates = 100)